<a href="https://www.kaggle.com/code/yutodennou/competition-catboost?scriptVersionId=181098104" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
%%capture
!pip install botorch

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from pathlib import Path

import optuna
from optuna.integration.botorch import BoTorchSampler

import sys
sys.path.append('/kaggle/input/winkler-interval-score-metric/')
import MWIS_metric

plt.style.use('ggplot')
plt.rcParams.update(**{'figure.dpi':150})

## Loading the data and preliminary cleaning

In [ ]:
data_path = Path('/kaggle/input/prediction-interval-competition-i-birth-weight')
train = pd.read_csv(data_path / 'train.csv', index_col=['id'])
test = pd.read_csv(data_path / 'test.csv', index_col=['id'])

train.head()

In [ ]:
print(f'Shape of training data (including output): {train.shape}')

In [ ]:
missing_codes = {
    'ATTEND': 9, 'BFACIL': 9,
    'BMI': 99.9, 'CIG_0': 99,
    'DLMP_MM': 99,
    'DOB_TT': 9999,
    'FAGECOMB': 99,
    'FEDUC': 9,
    'ILLB_R': 999, 'ILOP_R': 999, 'ILP_R': 999,
    'MBSTATE_REC': 3,
    'MEDUC': 9,
    'M_Ht_In': 99,
    'NO_INFEC': 9, 'NO_MMORB': 9, 'NO_RISKS': 9,
    'PAY': 9, 'PAY_REC': 9,
    'PRE_CARE': 99,
    'PREVIS': 99, 
    'PRIORDEAD': 99, 'PRIORLIVE': 99, 'PRIORTERM': 99,
    'PWgt_R': 999,
    'RDMMETH_REC': 999,
    'RF_CESARN': 99,
    'WTGAIN': 99
}

values = {column:np.nan for column in missing_codes}
# use pd.NA for certain categorical columns
categorical_cols_w_missing = [
    'ATTEND', 'BFACIL', 'LD_INDL', 'PAY, ''PAY_REC',
    'MBSTATE_REC', 'RDMETH_REC'
]

for column in categorical_cols_w_missing:
    if column in values:
        values[column] = pd.NA

In [ ]:
train = train.replace(to_replace=missing_codes, value=values)
test = test.replace(to_replace=missing_codes, value=values)

## Target variable to predict

In [ ]:
fig, (ax_box, ax_hist) = plt.subplots(
    2, figsize=(5, 4), sharex=True, gridspec_kw={"height_ratios": (.1, .9)}
)

_ = sns.boxplot(data=train, x='DBWT', ax=ax_box)
_ = sns.histplot(data=train, x='DBWT', kde=True, ax=ax_hist)

_ = ax_box.set(yticks=[], xlabel=None)

fig.tight_layout()

## Feature Analysis



### Missing values



In [ ]:
from numbers import Number

def filter_greater_than(s:pd.Series, threshold:Number) -> pd.Series:
    return s[s > threshold]

missing_perc_train = (
    (train.isna().sum() / train.shape[0] * 100)
    .pipe(filter_greater_than, threshold=0)
    .sort_values(ascending=False)
    .round(3)
)

missing_perc_train

### Mode-dominant features


In [ ]:
def get_mode_fraction(x:pd.Series) -> float:
    cts = x.value_counts(sort=True, ascending=False)
    return cts.iloc[0] / x.shape[0]

high_mode_per = (
    train.drop('DBWT', axis=1)
    .apply(get_mode_fraction)
    .pipe(filter_greater_than, threshold=0.9)
)

high_mode_per

In [ ]:
# drop these columns
train = train.drop(high_mode_per.index.tolist(), axis=1)
test = test.drop(high_mode_per.index.tolist(), axis=1)
print(f'Number of features after dropping: {train.shape[1] - 1}')

### Redundant features 


In [ ]:
train = train.drop(
    ['PAY', 'RF_CESAR'],
    axis=1
)

test = test.drop(
    ['PAY', 'RF_CESAR'],
    axis=1
)

### Categorical features


In [ ]:
categorical_cols = [
    'ATTEND', 'DMAR', 'LD_INDL', 'PAY_REC',
    'MBSTATE_REC', 'RDMETH_REC', 'RESTATUS',
    'SEX'
]

for col in categorical_cols:
    train[col] = train[col].astype('str')
    test[col] = test[col].astype('str')

In [ ]:
n_rows = 2
n_cols = 4
fig, axs = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 3 * n_rows), sharey=True)

for i in range(n_rows):
    for j in range(n_cols):
        idx = n_cols * i + j
        _ = sns.boxplot(
            train, 
            x= categorical_cols[idx],
            y='DBWT',
            ax=axs[i, j]
        )

fig.tight_layout()

### Correlation analysis - hierarchical clustering


In [ ]:
# hierarchical clustering
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

# construct correlation matrix
corr_matrix = (
    train.select_dtypes('number')
    .drop('DBWT', axis=1)
    .corr(method='spearman')
)

# hierarchical cluster based on the correlations
dissimilarity = 1 - abs(corr_matrix.values)

In [ ]:
linkage_matrix = hierarchy.linkage(squareform(dissimilarity), method='complete')

# plot the dendogram
fig,ax = plt.subplots(1,1,figsize=(7,5))
dendogram = hierarchy.dendrogram(
    linkage_matrix, ax=ax, 
    labels=corr_matrix.columns.tolist(),
)
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=75, fontsize=8)
_ = ax.set_ylabel('Distance')
_ = ax.set_title('Hierarchical clustering dendogram - spearman corr')
fig.tight_layout()

In [ ]:
train = train.drop(['PRIORTERM', 'BMI'], axis=1)
test = test.drop(['PRIORTERM', 'BMI'], axis=1)

### Feature engineering


In [ ]:
def add_duration(df:pd.DataFrame):
    df['preg_dur_approx'] = df['DOB_MM'] - df['DLMP_MM']

    # negative durations don't make sense. 
    # Credit: https://www.kaggle.com/code/paddykb/lgbm-mapie-birth-weight-oh-my
    df['preg_dur_approx'] = np.where(
        df['preg_dur_approx'] < 0,
        df['preg_dur_approx'] + 12,
        df['preg_dur_approx']
    )
    
add_duration(train)
add_duration(test)

## Preparing the data



In [ ]:
X = train.drop('DBWT', axis=1)
y = train['DBWT']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=1
)

## Tuning the multi-quantile catboost model


In [ ]:
import joblib
import warnings

def optuna_objective(trial:optuna.trial.Trial) -> float:
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 3000, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 15e-2, 0.75,log=True),
        'depth': trial.suggest_int('depth', 6, 9, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg',1e-8,100,log=True),
        'model_size_reg': trial.suggest_float('model_size_reg',1e-8,100,log=True),
        'random_strength': trial.suggest_float('random_strength',1e-8,100,log=True),
        'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 1),
        'subsample': trial.suggest_float("subsample", 0.1, 1)
    }
    
    alpha = 0.05 - 0.01
    quantile_levels = [alpha, 1 - alpha]
    quantile_str = str(quantile_levels).replace('[','').replace(']','')

    model = CatBoostRegressor(
        loss_function=f'MultiQuantile:alpha={quantile_str}',
        thread_count= 4,
        cat_features= categorical_cols,
        bootstrap_type =  "Bernoulli",
        sampling_frequency= 'PerTree',
        **params
    )
    
    # further split data into training and validation splits
    X_train2, X_valid2, y_train2, y_valid2 = train_test_split(
        X_train, y_train, random_state=4
    )
    
    # train model
    model.fit(X_train2, y_train2, verbose=0)
    
    # get predictions
    valid2_pred = model.predict(X_valid2)
    
    # get perfomance metrics
    MWIS, coverage = MWIS_metric.score(
        y_valid2, valid2_pred[:, 0], valid2_pred[:, 1], alpha=0.1
    )
    
    # store coverage results
    trial.set_user_attr("coverage", coverage)
    
    
    return MWIS

def coverage_constraints(trial):
    return (0.9 - trial.user_attrs['coverage'],)

with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=optuna.exceptions.ExperimentalWarning)
    warnings.simplefilter(action="ignore", category=FutureWarning)

    # create optuna study
    study = optuna.create_study(
        directions=['minimize'], 
        sampler=BoTorchSampler(n_startup_trials=9, seed=2, constraints_func=coverage_constraints),
        study_name='catboost'
    )

    # run optuna for a maxmimum of 100 trials and 1hr 30mins wall clock time
    study.optimize(optuna_objective, n_trials=100, timeout=5400) 

# save the runs
_ = joblib.dump(study, 'catboost_hyperopt_birthweight.pkl')
    

In [ ]:
fig = optuna.visualization.plot_optimization_history(study, target_name='CV MAE')

fig.update_layout(
    autosize=True,
    width=800,
    height=600
)
fig.show()

In [ ]:
results = study.trials_dataframe(attrs=('number','value', 'duration', 'params', 'user_attrs'))
results = results.rename(columns={'value':'MWIS'})
results['duration'] = results['duration']/np.timedelta64(1, 's')
results = results.sort_values(by='MWIS',ascending=True)
results.to_csv('cv_mwis_history.csv',index=False)

In [ ]:
study.best_params

## Final Catboost model

In [ ]:
alpha = 0.05 - 0.01
quantile_levels = [alpha, 1 - alpha]
quantile_str = str(quantile_levels).replace('[','').replace(']','')

model = CatBoostRegressor(
    loss_function=f'MultiQuantile:alpha={quantile_str}',
    thread_count= 4,
    cat_features= categorical_cols,
    bootstrap_type =  "Bernoulli",
    sampling_frequency= 'PerTree',
    **study.best_params
)
_ = model.fit(X_train, y_train, verbose=50)

In [ ]:
# evaluate on validation set
y_valid_predict = model.predict(X_valid)

predictions = y_valid.to_frame(name="y_true") # the "ground truth" column
predictions["pi_lower"] = y_valid_predict[:, 0]
predictions["pi_upper"] = y_valid_predict[:, 1]

alpha = 0.1 # the competition alpha
MWIS,coverage = MWIS_metric.score(predictions["y_true"],predictions["pi_lower"],predictions["pi_upper"],alpha)

print(f"Local MWI score........: {MWIS:.3f}",)
print(f"Predictions coverage...: {(coverage * 100):.2f}%")

## Test predictions

In [ ]:
test_pred = model.predict(test)

submission = pd.DataFrame({
    'id': test.index.tolist(),
    'pi_lower': test_pred[:, 0],
    'pi_upper': test_pred[:, 1]
})

submission.to_csv('submission.csv', index=False)